In [2]:
!git clone https://github.com/Vibha27/image-restoration-enhancement.git

Cloning into 'image-restoration-enhancement'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 54 (delta 10), reused 54 (delta 10), pack-reused 0
Unpacking objects: 100% (54/54), done.


In [28]:
import os, shutil
dir = r"/content/image-restoration-enhancement"
for folder in os.listdir(dir):
    src = os.path.join(dir, folder)
    if os.path.isdir(src):
        shutil.copytree(src, f"/content/{folder}")

!rm -r /content/static/uploads/*.jpg /content/static/uploads/*.png /content/static/uploads/*.jpeg

rm: cannot remove '/content/static/uploads/*.jpg': No such file or directory


In [4]:
 !pip install -qU flask-ngrok

# esrgan_tf model
# https://drive.google.com/file/d/14twK6Xm_yuL14eTjQbH3aynXDAXPbMLq/view?usp=sharing
!gdown --id 14twK6Xm_yuL14eTjQbH3aynXDAXPbMLq
!unzip -qq esrgan_tf.zip

Downloading...
From: https://drive.google.com/uc?id=14twK6Xm_yuL14eTjQbH3aynXDAXPbMLq
To: /content/esrgan_tf.zip
17.4MB [00:00, 33.8MB/s]


In [27]:
#@title Super_resolution module: Current Tensorflow
from PIL import Image
import numpy as np
import tensorflow as tf

# MODEL LOADS

super_res_model = tf.keras.models.load_model("/content/content/super_model/esrgan")
super_res_model.compile=False



# SUPER_RES PREPROCESSING TENSORFLOW

def preprocess_image(lr_image):
    if lr_image.shape[-1] == 4:
        lr_image = lr_image[...,:-1]

    lr_image = tf.cast(lr_image, tf.float32)

    lr_image = tf.convert_to_tensor(
        lr_image, dtype=None, dtype_hint=None, name=None
    )
    return tf.expand_dims(lr_image, 0)


def save_image(image, filepath):
    if not isinstance(image, Image.Image):
        image = tf.clip_by_value(image, 0, 255)
        image = Image.fromarray(tf.cast(image, tf.uint8).numpy())   
        
    image.save(filepath)


def super_resolution_module(name, file_path, save_dir):
    og_image = Image.open(file_path).convert("RGB")
    og_image = np.asarray(og_image)
    image = preprocess_image(og_image) 
    fake_image = super_res_model(image)
    fake_image = tf.squeeze(fake_image)
    fake_image = tf.clip_by_value(fake_image, 0, 255)
    fake_image = Image.fromarray(tf.cast(fake_image, tf.uint8).numpy()) 
    name = "out_"+name
    save_path = os.path.join(save_dir, name)
    save_image(fake_image, save_path)
    return save_path

# # TEST
# super_resolution_module("000060.png", r"/content/000060.png", r"/content/static/OUTPUTS")

In [29]:
import os
from flask import Flask, render_template, url_for, request, redirect, abort, send_from_directory
from flask_ngrok import run_with_ngrok
import time

SAVE_DIR = r"/content/static/OUTPUTS"
# upload_dir =r " "

!mkdir -p {SAVE_DIR}

app = Flask(__name__)
app.config['UPLOAD_EXTENSIONS'] = ['.jpeg', '.png', '.jpg']
app.config['UPLOAD_PATH'] = r'/content/static/uploads'
app.config['OUTPUT_PATH'] = r"/content/static/OUTPUTS"


@app.route('/')
def index() :
    return render_template('index.html')

@app.route('/upload-image/<pathname>')
def uplaod_image(pathname) :
    path = request.path
    module_name = path.split('/')[-1]
    return render_template('upload.html', pathname=module_name)

@app.route('/upload-image/<pathname>', methods=['POST'])
def upload_files(pathname):
    
    path = request.path
    module_name = path.split('/')[-1]
    uploaded_file = request.files['file']
    file_name = uploaded_file.filename
    file_path = os.path.join(r"/content/static/uploads", file_name)
    # output_file = request.files['file']

    
    if file_name != '':
        file_ext = os.path.splitext(file_name)[1]
        if file_ext not in app.config['UPLOAD_EXTENSIONS'] :
            abort(400)

        uploaded_file.save(os.path.join(app.config['UPLOAD_PATH'], file_name))
        # output_file.save(os.path.join(app.config['OUTPUT_PATH'], filename))

        # return filename
    
    print(module_name, file_path)
    if module_name == "super-resolution":
        test = os.path.exists(file_path)
        if test:
            print(file_name, file_path)
            processed_image_path = super_resolution_module(file_name, file_path, app.config['UPLOAD_PATH'])
        
    # elif module_name == "colorization":
    #     pass
    
    # elif module_name == "dehaze":
    #     pass
    
    # elif module_name == "inpaint":
    #     pass
    return redirect(url_for('uploaded_file', pathname=pathname, filename=file_name))

@app.route('/uploads/<filename>')
def send_file(filename):
    # print("SEND FILE")
    return send_from_directory(app.config['UPLOAD_PATH'], filename)

@app.route('/uploads/<filename>')
def send_output_file(filename):
    # while not os.path.join(app.config['OUTPUT_PATH'], filename):
    #     time.sleep(2)
    print("Found Rendering")
    return send_from_directory(r"/content/static/OUTPUTS", filename)


@app.route('/upload-image/<pathname>/<filename>')
def uploaded_file(pathname,filename):
    # filename = 'http://127.0.0.1:5000/uploads/' + filename
    return render_template('upload.html',pathname=pathname,filename=filename)

if __name__ == "__main__" :
    run_with_ngrok(app)
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://f695be91eef3.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Apr/2021 19:02:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:02:46] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:02:47] "GET /static/images/sr.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:02:47] "GET /static/images/inpaint.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:02:47] "GET /static/images/color.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:02:47] "GET /static/images/sr-output.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:02:47] "GET /static/images/dehaze.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:02:47] "GET /static/images/color-output.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:02:47] "GET /static/images/In-paint_1.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:02:47] "GET /static/images/haze-output.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:02:49] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2021 19:02:50] "GET /upload-image/super-resolution HTTP/1.1" 200 -
127.0.0.1 - -

super-resolution /content/static/uploads/0284.png
0284.png /content/static/uploads/0284.png


127.0.0.1 - - [14/Apr/2021 19:03:02] "POST /upload-image/super-resolution HTTP/1.1" 302 -
127.0.0.1 - - [14/Apr/2021 19:03:02] "GET /upload-image/super-resolution/0284.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:03:03] "GET /uploads/0284.png HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2021 19:03:03] "GET /static/uploads/out_0284.png HTTP/1.1" 200 -


SEND FILE
